<a href="https://colab.research.google.com/github/IverMartinsen/ColabNotebooks/blob/main/combined_model_test_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/src/Python')

import pandas as pd
import tensorflow as tf
import numpy as np
from modules.stratified_idxs import stratified_idxs
from modules.losses import MeanSquaredErrorKLD

In [ ]:
# Load dataframe of features
df = pd.read_csv(
    '/content/drive/MyDrive/Data/Grønlandskveiteotolitter/dataframe.csv')

# Locate data with complete set of features
notna = np.all(np.array(df.notna()), axis = 1)

# Drop data with incomplete set of features
df = df.dropna()

# Create stratified indices for selecting datasets for training etc.
train_idx, valid_idx, test_idx = stratified_idxs(
    df['age'], (0.6, 0.2, 0.2), seed=123)

# Create stratified subsets for training etc.
df_train = df.iloc[train_idx]
df_valid = df.iloc[valid_idx]
df_test = df.iloc[test_idx]

# Create utility functions
df_features = lambda df: (
    tf.convert_to_tensor(df['length']), tf.convert_to_tensor(df['sex'])) 

df_labels = lambda df: tf.convert_to_tensor(df['age'])

labels = np.array(df['age'])

In [ ]:
# Set constants
image_size = (128, 128)
batch_size = 32
num_splits = 5

file_path = '/content/drive/MyDrive/Data/Grønlandskveiteotolitter/greenland_halibut_std/'

# Load images from directory in alphabetical order
dataset = tf.keras.utils.image_dataset_from_directory(
    file_path,
    labels=None,
    image_size=image_size,
    shuffle=False)

# Stack images into numpy array
# Only use images with complete corresponding features
images = np.stack(list(dataset.unbatch().as_numpy_iterator()))[notna]

# List filenames
filenames = np.array(tf.io.gfile.listdir(file_path + 'images/'))

# training data
x_tr = images[train_idx]
y_tr = labels[train_idx]
f_tr = list(filenames[train_idx])

# validation data
x_va = images[valid_idx]
y_va = labels[valid_idx]
f_va = list(filenames[valid_idx])

# test data
x_te = images[test_idx]
y_te = labels[test_idx]
f_te = list(filenames[test_idx])

In [ ]:
# Create normalization layer for normalizing the numerical input.
Normalization = tf.keras.layers.Normalization(None)
# Fit normalization layer on training data
Normalization.adapt(df_train['length'])

# Create layer for mapping categorical labels to int
Index = tf.keras.layers.StringLookup()
# Fit index layer on training data
Index.adapt(tf.constant(df_train['sex']))

# Create layer for one-hot-encoding the categorical labels
Encoding = tf.keras.layers.CategoryEncoding(
    num_tokens=Index.vocabulary_size(), output_mode='one_hot')

In [ ]:
# Define model feature model
num_input = tf.keras.Input(shape=(1,), name='length')
cat_input = tf.keras.Input(shape=(1,), name='sex', dtype='string')

mlp_inputs = [num_input, cat_input]
features = tf.keras.layers.concatenate(
    [Normalization(num_input), Encoding(Index(cat_input))])
z = tf.keras.layers.Dense(8)(features)
z = tf.keras.layers.BatchNormalization()(z)
z = tf.keras.layers.ReLU()(z)
mlp_outputs = tf.keras.layers.Dense(1, 'relu')(z)

mlp = tf.keras.Model(mlp_inputs, mlp_outputs)

In [ ]:
# Define pretrained base model without classification head
base_model = tf.keras.applications.Xception(
    input_shape=image_size + (3, ), 
    include_top=False,
    pooling='avg')

# Freeze all layers in base model
#for layer in base_model.layers:
#    layer.trainable=False

# Define CNN. Note that by setting training=False in the base model
# we always run the model in inference mode 
cnn_inputs = tf.keras.layers.Input(image_size + (3, ))
x = tf.keras.applications.xception.preprocess_input(cnn_inputs)
x = tf.keras.layers.RandomTranslation(0, 0.1)(x)
x = tf.keras.layers.RandomRotation(0.1, fill_mode='constant')(x)
x = base_model(cnn_inputs, training=False)
x = tf.keras.layers.Dropout(0.2)(x)
cnn_outputs = tf.keras.layers.Dense(1, 'relu')(x)

cnn = tf.keras.models.Model(cnn_inputs, cnn_outputs)

In [ ]:
# Define combined model
combined = tf.keras.layers.concatenate([mlp.output, cnn.output])
outputs = tf.keras.layers.Dense(1, 'relu')(combined)

combined_model = tf.keras.Model([mlp.inputs, cnn.inputs], outputs)

In [ ]:
combined_model.compile(
    tf.keras.optimizers.Adam(1e-3), 
    tf.keras.losses.MeanSquaredError())

callbacks = tf.keras.callbacks.EarlyStopping(
    patience=20, restore_best_weights=True)

combined_model.fit([df_features(df_train), x_tr],
                   y_tr,
                   batch_size=32,
                   epochs=100,
                   validation_data=([df_features(df_valid), x_va], y_va),
                   callbacks = callbacks)

In [ ]:
combined_model.evaluate(([df_features(df_test), x_te]), y_te)

In [ ]:
# Create layer for mapping categorical labels to int
Index = tf.keras.layers.StringLookup()
# Fit index layer on training data
Index.adapt(tf.constant(df_train['sex']))

#image_size = (128, 128)

# Create layer for one-hot-encoding the categorical labels
Encoding = tf.keras.layers.CategoryEncoding(
    num_tokens=Index.vocabulary_size(), output_mode='one_hot')

# Define pretrained base model without classification head
base_model = tf.keras.applications.Xception(
    input_shape=image_size + (3, ), 
    include_top=False,
    pooling='avg')

# Define CNN. Note that by setting training=False in the base model
# we always run the model in inference mode 
cnn_input = tf.keras.layers.Input(image_size + (3, ))
cat_input = tf.keras.Input(shape=(1,), name='gender', dtype='string')
gender = Encoding(Index(cat_input))

x = tf.keras.applications.xception.preprocess_input(cnn_input)
x = tf.keras.layers.RandomTranslation(0, 0.1)(x)
x = tf.keras.layers.RandomRotation(0.1, fill_mode='constant')(x)
x = base_model(x, training=False)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(3, 'relu')(x)
outputs = tf.keras.layers.Dot(axes=1)([x, gender])

cnn = tf.keras.models.Model([cat_input, cnn_input], tf.keras.layers.Concatenate()([outputs, gender]))

In [ ]:
def normal_divergence(mean1, mean2, sigma1, sigma2):

    sigma2 = tf.cast(tf.constant(1, shape=sigma2.shape), sigma2.dtype)

    div = tf.math.log(sigma2 / sigma1) + (sigma1 ** 2 + (mean1 - mean2) ** 2) / (2 * sigma2 ** 2) - 1 / 2
    
    return div
    #return tf.where(tf.math.is_nan(div), div, tf.zeros_like(div))
    #return tf.zeros_like(div)


class MeanSquaredErrorKLD(tf.keras.losses.Loss):


    def call(self, y_true, y_pred):
        
        mask = y_pred[:, 2:]
        tmp = tf.reshape(y_pred[:, 0], (-1, 1))
        
        _y_true = tf.cast(y_true*tf.cast(mask, y_true.dtype), tf.float64)
        _y_pred = tf.cast(tmp*tf.cast(mask, tmp.dtype), tf.float64)
        
        n_true = tf.reduce_sum(tf.cast(_y_true != 0, _y_true.dtype), axis=0)
        n_pred = tf.reduce_sum(tf.cast(_y_pred != 0, _y_pred.dtype), axis=0)

        mean_true = tf.reduce_sum(_y_true, axis=0) / n_true
        mean_pred = tf.reduce_sum(_y_pred, axis=0) / n_pred

        std_true = tf.reduce_sum((_y_true - mean_true*tf.cast(mask, _y_true.dtype))**2, axis=0) / n_true
        std_pred = tf.reduce_sum((_y_pred - mean_pred*tf.cast(mask, _y_true.dtype))**2, axis=0) / n_pred

        gamma = tf.constant(1, dtype=tf.float64)

        kld = gamma*tf.reduce_sum(normal_divergence(mean_true, mean_pred, std_true, std_pred))
        
        return tf.keras.losses.mean_squared_error(y_true, tmp) + tf.cast(kld, tf.float32)


In [ ]:
cnn.compile(
    tf.keras.optimizers.Adam(1e-3), 
    MeanSquaredErrorKLD())

callbacks = tf.keras.callbacks.EarlyStopping(
    patience=20, restore_best_weights=True)

cnn.fit([tf.convert_to_tensor(df_train['sex']), x_tr],
                   y_tr,
                   batch_size=32,
                   epochs=100,
                   validation_data=([tf.convert_to_tensor(df_valid['sex']), x_va], y_va),
                   callbacks = callbacks)

Epoch 1/100
67/67 [==============================] - 37s 461ms/step - loss: 287.5089 - val_loss: 251.2334
Epoch 2/100
66/67 [============================>.] - ETA: 0s - loss: 259.1676

KeyboardInterrupt: ignored

In [ ]:
tf.reshape(cnn([tf.convert_to_tensor(df_test['sex'].iloc[:10, ]), x_te[:10]])[:, 0], (-1, 1))

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[12.236333 ],
       [10.335702 ],
       [12.2605095],
       [12.243863 ],
       [12.256292 ],
       [10.328697 ],
       [12.252926 ],
       [10.331232 ],
       [10.304781 ],
       [12.256765 ]], dtype=float32)>

In [ ]:
y_te[:10]

array([14, 11, 12, 12, 16,  9, 13, 11,  4, 12])

In [ ]:


cnn.evaluate([tf.convert_to_tensor(df_test['sex'].iloc[idx]), x_te[idx]], y_te[idx])

NameError: ignored

In [ ]:
idx = np.where(df_test['sex'] == 'male')
df_test['sex'].iloc[idx]

In [ ]:
np.sum(np.abs(y_hat[idx] - y_te[idx]) == 0) / len(y_hat[idx])

In [ ]:
y_hat = np.array(cnn.predict([tf.convert_to_tensor(df_test['sex']), x_te]).round()).reshape(-1)

In [ ]:
cnn.save('/content/drive/MyDrive/Forberedende forsøk/testrun_5_xception')

In [ ]:
df = pd.DataFrame({'true_age': y_te, 'predicted_age': y_hat})
df.to_csv('loss5.csv', index=False)